# Graph three-colorability

In this chapter we construct a zero-knowledge protocol around graph colorability.

# What is a graph? What is colorability?

See the chapter on graph isomorphism for what a **graph** is.

**Coloring** a graph means to assign a color to each node. For each edge, adjacent nodes must have a different color.

A coloring comes with a number of colors. How many colors we have available determines how hard it is to color a graph.

For instance, [it was proven that every graph can be four-colored](https://en.wikipedia.org/wiki/Four_color_theorem). In contrast, not every graph can be three-colored! In fact, three-coloring is an NP-complete problem. This means that for large graphs it may take exponential time to three-color them.

A graph is **three-colorable** if there is a three-coloring for it.

# What is an interactive proof?

Peggy and Victor are engaged in an interactive proof.

There is a (random) graph.

Peggy thinks she knows a three-coloring of the graph. She wants to prove that to Victor, without revealing the coloring.

Victor is sceptical and wants to see evidence. In particular he doesn't want to accept if Peggy is lying (there is no coloring / the coloring is invalid).

Peggy wins if she convinces Victor. Victor wins by accepting only graphs that are actually three-colorable.

# Graph parameters

Choose the number of nodes of the graph.

Feel free to experiment with different values.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# You can adjust the slider any time

num_nodes = widgets.IntSlider(min=2, max=20, value=2, step=1, description="#Nodes")
num_nodes

# Scenario

There are two scenarios in this interactive proof:

1. **Peggy is honest** and knows a three-coloring of the graph. She hopes to convince Victor of the true statement that the graph is three-colorable.
2. **Peggy is lying** and doesn't know anything! The graph is in fact not three-colorable! Peggy still tries to fool Victor and convince him of the false statement that the graph is three-colorable.

Choose the good or the evil scenario. See how it affects the other cells further down.

In [ ]:
# You can adjust the selection any time
# Make sure to re-generate the graph afterwards

is_colorable = widgets.Dropdown(
    options=[
        ("Graph is three-colorable (honest Peggy 😇)", True),
        ("Graph is not three-colorable (lying Peggy 😈)", False)],
    value=True,
    description="Scenario:",
)
is_colorable

# Graph

Now generate the graph. In the good scenario it is three-colorable. In the evil scenario it is not three-colorable.

You will see the colors in the plot.

In [ ]:
# Rerun this cell to generate a new graph
# Rerun this cell after changing the scenario above

import networkx as nx
import matplotlib.pyplot as plt
from graph import Mapping, three_colorable_graph, not_three_colorable_graph

if is_colorable.value:
    graph, coloring = three_colorable_graph(num_nodes.value)
else:
    graph, coloring = not_three_colorable_graph(num_nodes.value)

print("Colored graph")
node_colors = [coloring[node] for node in graph.nodes]
nx.draw(graph, node_color=node_colors, with_labels=True)
plt.show()

# Protocol

Here is an interactive proof for three coloring.

To hide her coloring, Peggy shuffles the colors around: Red becomes blue, blue becomes green, green becomes red; or something similar. There are six distinct ways how three colors can be shuffled.

Victor selects a random edge and challenges Peggy to color it.

Peggy colors the edge, but using her shuffled colors.

Victor checks if the edge is correctly colored, i.e., the colors are different.

The shuffling hides the original colors from Victor, but with the current protocol Peggy might return two different random colors without problem. We solve this by using commitments:

After Peggy shuffled her colors, she commits to the color of each edge. She sends these commitments (curve points) to Victor. Later when Peggy colors Victor's edge, she opens the commitments to the nodes of this edge. Victor checks that the original commitments match the openings.

If Peggy's coloring is invalid, then both nodes of one edge have the same color. There is a chance that Victor randomly chooses this edge. Peggy cannot open the commitments to other colors that the original ones. Victor will see that both colors are the same.

In [ ]:
from ec.static import ONE_POINT, random_point, Scalar, CurvePoint
from ec.util import Opening
from typing import List, Tuple, Dict
import random

punto_uno = random_point()
assert ONE_POINT != punto_uno, "This may fail due to randomness; run the cell again"

Edge = Tuple[int, int]
ColoredEdge = Tuple[Opening, Opening]

class Peggy:
    def __init__(self, coloring: Dict[int, int]):
        self.coloring = Mapping(coloring)
        
    def commit(self):
        shuffle = Mapping.shuffle_list([0, 1, 2])
        self.shuffled_coloring = self.coloring.and_then(shuffle)
        
        self.openings = []
        commitments = []
        
        for index in range(len(self.coloring)):
            opening = Opening(Scalar(self.shuffled_coloring[index]), ONE_POINT, punto_uno)
            self.openings.append(opening)
            commitments.append(opening.close())
            
        return commitments
    
    def color_edge(self, edge: Edge) -> ColoredEdge:
        left_color_opening = self.openings[edge[0]]
        right_color_opening = self.openings[edge[1]]
        
        return left_color_opening, right_color_opening

class Victor:
    def __init__(self, edges):
        self.edges = edges
    
    def random_edge(self, commitments: List[CurvePoint]) -> Edge:
        self.commitments = commitments
        self.edge = random.choice(self.edges)
        
        return self.edge
    
    def verify(self, colored_edge: ColoredEdge) -> bool:
        # Invalid coloring
        if colored_edge[0].value() == colored_edge[1].value():
            return False
        # Openened colors correspond to commitments
        if not colored_edge[0].verify(self.commitments[self.edge[0]]):
            return False
        if not colored_edge[1].verify(self.commitments[self.edge[1]]):
            return False
        
        return True

# Single Exchange

Let's run a couple of exchanges to get familiar. Feel free to experiment with the graph parameters and the scenario. 

The system is random so there could be a different outcome next time you run with the same settings 😉

In [ ]:
# Feel free to rerun this multiple times!

peggy = Peggy(coloring)
victor = Victor(graph)

commitments = peggy.commit()
print("Node color commitments: {}".format(commitments))

edge = victor.random_edge(commitments)
print("Edge: {}".format(edge))

colored_edge = peggy.color_edge(edge)
print("Colored edge: {}".format(colored_edge))
print()

# Victor is convinced
if victor.verify(colored_edge):
    # Graph is three-colorable (good)
    if is_colorable.value:
        print("Convinced 👌 (expected)")
    # Graph is not three-colorable (evil)
    else:
        print("Convinced 👌 (Victor was fooled)")
# Victor is not convinced
else:
    # Graph is three-colorable (good)
    if is_colorable.value:
        print("Not convinced... 🤨 (Peggy was dumb)")
    # Graph is not three-colorable (evil)
    else:
        print("Not convinced... 🤨 (expected)")

# Completeness

Does Peggy have a chance to convince Victor if she is honest? Let's run the protocol a couple of times and see how often she manages to convince him.

In [ ]:
num_exchanges_complete = widgets.IntSlider(min=10, max=1000, value=10, step=10, description="#Exchanges")
num_exchanges_complete

In [ ]:
# Honest case
graph2, coloring2 = three_colorable_graph(num_nodes.value)

honest_peggy = Peggy(coloring2)
victor = Victor(graph2)

peggy_success = 0

for _ in range(num_exchanges_complete.value):
    commitments = honest_peggy.commit()
    edge = victor.random_edge(commitments)
    colored_edge = honest_peggy.color_edge(edge)

    if victor.verify(colored_edge):
        peggy_success += 1
        
peggy_success_rate = peggy_success / num_exchanges_complete.value * 100

print("Running {} exchanges".format(num_exchanges_complete.value))
print("Honest Peggy wins {0:0.2f}% of the time".format(peggy_success_rate))
print()

assert peggy_success_rate == 100
print("Peggy always wins if she is honest")

# Soundness

Now, does Victor have a chance to expose Peggy if she is lying?

Depending on the graph, his chances of choosing an edge that breaks the coloring might be pretty slim. If Peggy's coloring of `n` nodes is valid except for one edge, then Victor has a `1 - 1/n` chance of being fooled, which is pretty bad.

But there is our usual trick: We run the protocol for **multiple rounds** to boost Victor's confidence. Peggy shuffles her colors and commits to them, Victor chooses a random edge, Peggy responds; repeat. Victor verifies that Peggy correctly colors his chosen edges. If Peggy fails only once, he rejects her completely. He accepts if Peggy meets all of his challenges.

The shuffled coloring is different every time, but the edges that are incorrectly colored are always the same. _(If we shuffle the colors of an edge with the same color at both ends, the colors will still be the same!)_ Every round, Victor has a chance of guessing a critical edge, so the total probability of choosing such an edge increases exponentially with the number of rounds.

In [ ]:
num_exchanges_sound = widgets.IntSlider(min=10, max=1000, value=10, step=10, description="#Exchanges")
num_rounds = widgets.IntSlider(min=1, max=15, value=1, step=1, description="#Rounds")

display(num_exchanges_sound)
display(num_rounds)

In [ ]:
# Lying case
graph3, coloring3 = not_three_colorable_graph(num_nodes.value)

lying_peggy = Peggy(coloring3)
victor = Victor(graph3)

victor_success = 0

for _ in range(num_exchanges_sound.value):
    for _ in range(num_rounds.value):
        commitments = lying_peggy.commit()
        edge = victor.random_edge(commitments)
        colored_edge = lying_peggy.color_edge(edge)
    
        if not victor.verify(colored_edge):
            victor_success += 1
            break
            
victor_success_rate = victor_success / num_exchanges_sound.value * 100

print("Running {} exchanges with {} rounds each".format(num_exchanges_sound.value, num_rounds.value))
print("Victor wins against lying Peggy {0:0.2f}% of the time".format(victor_success_rate))
print()

if victor_success_rate < 50:
    print("Victor loses quite often for a small number of rounds")
elif victor_success_rate < 90:
    print("Victor gains more confidence with each added round")
else:
    print("At some point it is basically impossible to fool Victor")

# Zero-Knowledge

Does Victor learn anything about the secret coloring from his exchange with Peggy?

Peggy sends commitments to her shuffled coloring. Victor sends an edge. Peggy sends openings to the colors of the edge. The opening consists of the color itself and the random blinding factor.

We can replicate this behavior by sending random colors. Peggy could use the same trick to fool Victor before we introduced commitments.

Commitments are binding forwards in time: Once Peggy commits to a value, she can no longer change it. She is forced to open the commitment to the same value. The value depends on the challenge of Victor, so she cannot know it in advance. This is what gives Victor confidence that Peggy was ready to answer any challenge, not just the one that he randomly chose.

For the fake transcripts, we can freely pick a challenge (an edge). Once we know that, we can create commitments to random colors for the nodes inside this edge. The other nodes get random curve points as commitments.

The resulting transcript looks the same because commitments are hiding. They are random curve points.

To summarize, we send manufactured commitments, a random edge and two random openings (random color and random blinding factor).

Let's do a chi-square test to see if the real and fake transcripts follow the same distribution.

In [ ]:
num_transcripts = widgets.IntSlider(min=1000, max=100000, value=10000, step=1000, description="#Transcripts")
num_transcripts

In [ ]:
from ec.static import random_scalar
from typing import Tuple
import stats

# Make sure to run this for small graphs (adjust the graph parameters)
# Large graphs can lead to errors
# The list of critical chi-square values might be too short for the large number of degrees of freedom
# Large graphs also require very many transcripts, which is slow

peggy = Peggy(coloring)
victor = Victor(graph)
edges = sorted(graph.edges())

def real_transcript() -> Tuple:
    commitments = peggy.commit()
    edge = victor.random_edge(commitments)
    left_open, right_open = peggy.color_edge(edge)

    return tuple(commitments), edge, left_open.serialize(), right_open.serialize()


def fake_transcript() -> Tuple:
    edge = random.choice(edges)
    left_node, right_node = edge
    
    left_color = random.randrange(0, 3)
    if random.random() > 0.5:
        right_color = (left_color + 1) % 3
    else:
        right_color = (left_color - 1) % 3
        
    left_color = Scalar(left_color)
    right_color = Scalar(right_color)
    
    left_open = Opening(left_color, ONE_POINT, punto_uno)
    right_open = Opening(right_color, ONE_POINT, punto_uno)
    commitments = [random_point() for _ in range(len(graph.nodes))]
    commitments[left_node] = left_open.close()
    commitments[right_node] = right_open.close()
        
    return tuple(commitments), edge, left_open.serialize(), right_open.serialize()

print("Real transcript: {}".format(real_transcript()))
print("Fake transcript: {}".format(fake_transcript()))
print()

real_samples = [real_transcript() for _ in range(num_transcripts.value)]
fake_samples = [fake_transcript() for _ in range(num_transcripts.value)]

# The chi-square test is only valid if most bins are filled
# Increase the number of transcripts if there are too many empty bins

null_hypothesis = stats.chi_square_equal(real_samples, fake_samples)
print()

if null_hypothesis:
    print("Real and fake transcripts are the same distribution.")
    print("Victor learns nothing 👌")
else:
    print("Real and fake transcripts are different distributions.")
    print("Victor might learn something 😧")